In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import allel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, cohen_kappa_score, f1_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline


import umap.umap_ as umap

In [5]:
R_SEED = 22
home = os.path.expanduser('~')
directory = os.path.join('Imp_Research','Dataset')

# populations = ['BFcol','BFgam','AOcol','CIcol','CMgam','FRgam',
#               'GAgam','GHcol','GHgam','GM','GNcol','GNgam','GQgam',
#               'GW','KE','UGgam']

# Using only 12 populations out of 16
populations = ['BFcol','BFgam','AOcol','CIcol','CMgam',
              'GAgam','GHcol','GM','GNgam','GW','KE','UGgam']

# Creating a dictionary of Population names with labels
populations_encoding = {}
for i in range(len(populations)):
    populations_encoding[i] = populations[i]

### Functions to Load and Pre-Process the Datasets

In [80]:
'''
Requirements : NumPy and Scikit-Allel
'''

class FilterSNP():
    def __init__(self,haplotype, POS):
        self.haplotype = haplotype
        self.POS = POS
        self.H = haplotype
        self.P = POS
        self._removed_maf = None
        self._retained_maf = None
        self._retained_ld = None
        self._removed_ld = None
        self._unlinked_POS = None
        
    def all_filters(self, LD_window_size, LD_overlap_step, MBP_start = 1, MBP_end = 37, MAF_threshold = 5, LD_threshold=.1,LD_iter=1):
        print("1.) Selecting Mega Base Pairs")
        self.H,self.P = self.get_haplo_MBP(self.H,self.P,start = MBP_start, end = MBP_end)
        print("MBP selected. Retained Matrix = ", self.H.shape)
        print("2.) Filtering Rare Allels")
        self.H,self.P = self.filter_MAF(self.H,self.P,threshold = MAF_threshold)
        print("3.) Performing LD Pruning")
        self.H,self.P = self.LD_pruning(self.H, self.P, LD_window_size, LD_overlap_step, threshold = LD_threshold, n_iter = LD_iter)
        print("Retained Haplotype Matrix = ", self.H.shape)
        print("Retained Positions Matrix = ",self.P.shape)
        
        return self.H, self.P
        
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def filter_MAF(self,haplo,POS,threshold = 5):
        if threshold >= 50 : 
            print("MAF threshold cannot be more than 49%")
            return
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = self.get_MAF(haplo)
        #indexes = np.where(maf >= threshold*0.01)[0]
        minor = samples*threshold/100
        major = samples*(100-threshold)/100
        # Selects indexes where allels are >threshold or all 0 and all 1.
        indexes = np.where((sums>=minor)& (sums<=major))[0]
        print("Number of SNPs removed = ",len(haplo)-len(indexes))
        print("Retaining = ",len(indexes))
        self._removed_maf = len(haplo)-len(indexes)
        self._retained_maf = len(indexes)
        return np.take(haplo,indexes,0), np.take(POS,indexes,0)

    # Returns : Array of MAF
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_MAF(self,haplo):
        samples = haplo.shape[1]
        sums = haplo.sum(axis=1)
        maf = []
        for s in sums:
            if s != samples or s != 0:
                frequency = s/samples
                if frequency > 0.5:
                    maf.append(1-frequency)
                else: 
                    maf.append(frequency)
        return np.array(maf)

    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_MBP(self,POS,start = 1,end = 37):
        return np.where(POS[np.where(POS>=1e6)]<=37e6)[0]

    # input dimensions for Haplotype matrix (gn) = SNP X n
    def get_haplo_MBP(self,haplotype,POS,start = 1,end = 37):
        index = self.get_MBP(POS,start,end)
        return np.take(haplotype,index,0),np.take(POS,index,0)
    
    # input dimensions for Haplotype matrix (gn) = SNP X n
    def LD_pruning(self, gn, pos, size, step, threshold = .1, n_iter=1):
        removed = 0
        for i in range(n_iter):
            
            # Returns a boolean array. True(1) - SNPs are within threshold (Unliked), False(0) = Linked
            loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
            
            # Counts non-zero elements = No. of unlinked loci or SNPs to retain
            n = np.count_nonzero(loc_unlinked)
            
            # Calculate the number of SNPs to be removed
            n_remove = gn.shape[0] - n
            removed += n_remove
            print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
            
            # Select only the unlinked SNPs, i.e., indexes where value is True(1).
            gn = gn.compress(loc_unlinked, axis=0)
            
            # retaining the indexes preserved
            pos = pos.compress(loc_unlinked)
        self._retained_ld = gn.shape[0]
        self._removed_ld = removed
        self._unlinked_POS = pos
        return gn,pos

In [7]:
'''
Disclaimer : The class is for personal use. It is not aimed for portability or reusability.

Class to load and filter the data from disk.
Object Parameters : 
data_path  -> Path to the data directory

load_pop() : Function to load the data
params : 
Populations -> list or array of population names/filenames
filtered -> boolean, whethere to filter the data or not
combine -> Boolean, to combine the populations (Implement it !!!)

returns : A dictionary of haplotype matrix and Position array
'''

class LoadFilteredPops():
    def __init__(self,data_path = None):
        self.data_path = data_path
        
    def load_pop(self,populations,naming='custom',chromo_arms = ['3R'],filtered = True):
        if self.data_path is None:
            home = os.path.expanduser('~')
            directory = os.path.join('Imp_Research','Dataset')
        Haplo_pop = {}
        POS_pop = {}
        for population in populations:
            for arm in chromo_arms:
                pop_name = population+'.'+arm
                if naming == 'custom':
                    filename = f'Haplotype.POS.{pop_name}.hd5'
                else:
                    filename = population
                if self.data_path is None:
                    data_path = os.path.join(home, directory,"HDF_Dataset", filename)
                else:
                    try:
                        data_path = os.path.join(self.data_path,filename)
                    except:
                        print("Cannot resolve Directory path")
                        exit()
                print(f'------{pop_name}------\n')
                H = pd.read_hdf(data_path,key='Haplotype').astype('int8').to_numpy().astype('int8')
                P = pd.read_hdf(data_path,key='POS').to_numpy()
                
                if filtered:
                    # Uses FilterSNP class for filtering steps
                    datafilter = FilterSNP(H,P)
                    Haplo_pop[pop_name],POS_pop[pop_name] = datafilter.all_filters(LD_window_size = 500,LD_overlap_step = 100,LD_iter = 3)
                else: 
                    Haplo_pop[pop_name],POS_pop[pop_name] = H,P
                del H,P     
        print("Populations loaded !!!")
        return Haplo_pop,POS_pop

    
'''
Function to combine the populations
Input Params
Haplo_all : Dictionary containing Haplotype matrix for populations. Key-> pop name; Value -> matrix.
            Matrix dimensions must be POS x haplotypes.
POS_all : Position array of SNPs
filtered : Boolean, to filter after combining or not.
get_labels : Boolean, to generate labels

returns
Haplo_all : ndarray of n x m dimensions. n = length of POS array; m = sum of haplotypes from all populations.
POS : SNP Position array
labels : list of labels if Labels = []
'''

# def all_filters(self, LD_window_size, LD_overlap_step, MBP_start = 1, 
#                 MBP_end = 37, MAF_threshold = 5, LD_threshold=.1,LD_iter=1):

def combine_pops(H_all,P_all,filtered = True,get_labels = True,LD_window_size = 500, LD_overlap_step = 100,
                 MBP_start = 1, MBP_end = 37, MAF_threshold = 5, LD_threshold=.1,LD_iter=3):
    keys = list(H_all.keys())
    
    # Unpacking the Haplotype matrices of all pops from dictionary
    H = np.array(H_all[keys[0]])
    if len(H_all) > 1:
        for i in range(1,len(H_all)):
            H = np.append(H,H_all[keys[i]],axis=1)
    print('Combined Shape => ',H.shape)
    # Generating Labels if True
    if get_labels:
        label = []
        for each in keys:
            label.extend([each]*len(H_all[each][0]))
                          
    # Returning the combined Haplotype data matrix if Filtered = False
    if filtered == False:
        if get_labels:
            return H,P_all,label
        else:
            return H,p_all
    else:
        # Uses FilterSNP class for filtering steps
        datafilter = FilterSNP(H,P_all)
        H_filtered,POS_filtered = datafilter.all_filters(LD_window_size,LD_overlap_step,MBP_start,MBP_end,
                                                            MAF_threshold,LD_threshold,LD_iter)
        print('Final Filtered Shape = ',H_filtered.shape)
        if get_labels:
            return H_filtered,POS_filtered,label
        else:
            return H_filtered,POS_filtered
        

### Loading Dataset

In [8]:
# Loading dataset into the memory using the custom loader classes above
# Stored into dictionaries

loader = LoadFilteredPops()
Haplo_all, POS_all = loader.load_pop(populations,filtered = False)

------BFcol.3R------

------BFgam.3R------

------AOcol.3R------

------CIcol.3R------

------CMgam.3R------

------GAgam.3R------

------GHcol.3R------

------GM.3R------

------GNgam.3R------

------GW.3R------

------KE.3R------

------UGgam.3R------

Populations loaded !!!


#### Downsampling CMgam

In [77]:
H = Haplo_all.copy()

# Find the indexes  of CMgam
cmgam = H['CMgam.3R']
# Retaining only first 100 samples (200 Haplotypes)
H['CMgam.3R'] = cmgam[:,:200]
print('Shape of CMgam: ',H['CMgam.3R'].shape)

Shape of CMgam:  (4836295, 200)


#### Converting Haplotypes into Genotypes

In [58]:
for key in H.keys():
    X = H[key].T
    X_g = []
    for i in range(0,len(X),2):
        X_g.append(X[i]+X[i+1])
    X_g = np.array(X_g)
    H[key] = X_g
    print(f'{key} transformed shape: \t{X_g.shape}')

BFcol.3R transformed shape: 	(75, 4836295)
BFgam.3R transformed shape: 	(92, 4836295)
AOcol.3R transformed shape: 	(78, 4836295)
CIcol.3R transformed shape: 	(71, 4836295)
CMgam.3R transformed shape: 	(100, 4836295)
GAgam.3R transformed shape: 	(69, 4836295)
GHcol.3R transformed shape: 	(55, 4836295)
GM.3R transformed shape: 	(65, 4836295)
GNgam.3R transformed shape: 	(40, 4836295)
GW.3R transformed shape: 	(91, 4836295)
KE.3R transformed shape: 	(48, 4836295)
UGgam.3R transformed shape: 	(112, 4836295)


### Analysis Script

In [73]:
def to_genotype(X):
    Xg = []
    for i in range(0,len(X),2):
        Xg.append(X[i]+X[i+1])
    return np.array(Xg)

def evaluate_classifier(y_true,y_pred):
    test_accuracy = accuracy_score(y_true,y_pred)
    f1w = f1_score(y_true,y_pred,average= 'weighted')
    kappa = cohen_kappa_score(y_true,y_pred)
    return test_accuracy, f1w, kappa

In [ ]:
'''
Pairwise analysis of all the 12 populations.
Total 66 combinations/analysis
Analysis includes:
1. Selecting a pair
2. Filtering the SNPs jointly
    - Mega Base Pair Selection
    - Rare Allele Filtering with MAF >= 5%
    - LD Pruning
    Store Filtering stats for all the pairs
3. Haplo to Geno and Train-Test Split
4. Classifying using Logistic Regression and LDA
5. Generating UMAP visualization
6. Dimensionality Reduction: Obtaining 20 UMAP components 
7. Training LR classifier on UMAP components
8. Dumping all the stats and evaluation scores into a dataframe
'''
# Storage Path
home = home = os.path.expanduser('~')
directory = os.path.join(home,'Imp_Research','Repository','Pairwise_Analysis')

# List of keys
keys = list(H.keys())
count = 0
pnames = []
psamples = []
pmaf = []
psnps_init = []
psnps_filt = []
accuracies_lr = []
f1scores_lr = []
kappas_lr = []
accuracies_ulr = []
f1scores_ulr = []
kappas_ulr = []


for i in range(11):
    for j in range(i+1,12):
        count+=1
        pair_pos = POS_all[keys[i]]
        # 1. Pair Selection. Combining the populations. 
        pop1 = H[keys[i]]
        pop2 = H[keys[j]]
        pair_h = np.append(pop1,pop2,1)
        pname = f'{keys[i][:-3]}_{keys[j][:-3]}'
        psnp_init = pair_h.shape[0]
        psample = pair_h.shape[1]
        print(f'{pname}\n{psnp_init}')
        
        # 2. Filtering (Using FilterSNP class) Taking window size 250 is quicker.
        fsnp = FilterSNP(pair_h,pair_pos)
        pair_h, pair_pos = fsnp.all_filters(LD_window_size = 2000,LD_overlap_step = 200,LD_iter = 3)
        psnp_filt = pair_h.shape[0]
        pmaf.append(fsnp._retained_maf)
#         ppos_filt = len(pair_pos)
        
        # 3. Haplo to Geno and Train-Test Split
        X = pair_h.T
        Xg = to_genotype(X)       #Genotype data
        pshape_g = Xg.shape
        Y = [0]*int(len(pop1[0])/2)
        Y.extend([1]*int(len(pop2[0])/2))
        Y = np.array(Y)
        x_train,x_test,y_train,y_test = train_test_split(Xg,Y,test_size = 0.8,stratify = Y,random_state= R_SEED)
        
        # 4. LR Classifier
        lr = LogisticRegression(n_jobs = -1)
        lr.fit(x_train,y_train)
        acc_lr, f1w_lr, kappa_lr = evaluate_classifier(y_test,lr.predict(x_test))
        
        # 5. Generate UMAP visualization
        UMAP_g = umap.UMAP(n_neighbors=15,metric='euclidean',min_dist=2,spread=5).fit_transform(Xg)
        sns.set(style='white')
        fig, ax = plt.subplots(1, figsize=(14, 10))
        plt.scatter(UMAP_g[:, 0],UMAP_g[:, 1],c=Y,cmap='coolwarm')
        plt.setp(ax, xticks=[], yticks=[])
        cbar = plt.colorbar(boundaries=np.arange(3)-0.5)
        cbar.set_ticks(np.arange(2))
        cbar.set_ticklabels(populations)
        plt.title(f'Genotype populations {pname} UMAP. metric = euclidean,neighbors = 15,min-dist = 2,spread = 5');
        plt.show()
        figname = f'{count}_{pname}_umap.png'
        path = os.path.join(directory,'pairwise_plots',figname)
        plt.savefig(path,dpi=300)
        
        # 6. Dimensionality Reduction: Obtaining 20 umap components
        UMAP_g = umap.UMAP(n_components = 20,n_neighbors=15,metric='euclidean',min_dist=2,spread=5).fit(x_train)
        x_train = UMAP_g.embedding_
        x_test = UMAP_g.transform(x_test)
        
        # 7. Training LR on UMAP components
        ulr = LogisticRegression(n_jobs = -1)
        ulr.fit(x_train,y_train)
        acc_ulr, f1w_ulr, kappa_ulr = evaluate_classifier(y_test,ulr.predict(x_test))
    
    pnames.append(pname)
    psamples.append(psample)
    psnps_init.append(psnp_init)
    psnps_filt.append(psnp_filt)
    accuracies_lr.append(acc_lr)
    f1scores_lr.append(f1w_lr)
    kappas_lr.append(kappa_lr)
    accuracies_ulr.append(acc_ulr)
    f1scores_ulr.append(f1w_ulr)
    kappas_ulr.append(kappa_ulr)
    
pairwise_analysis = pd.DataFrame()
pairwise_analysis['Count'] = range(1,len(pnames)+1)
pairwise_analysis['Population_pair'] = pnames
pairwise_analysis['Samples'] = psamples
pairwise_analysis['Initial_SNP'] = psnps_init
pairwise_analysis['MAF_filter_SNP'] = pmaf
pairwise_analysis['Filtered_SNP'] = psnps_filt
pairwise_analysis['LR_accuracy'] = accuracies_lr
pairwise_analysis['LR_f1score'] = f1scores_lr
pairwise_analysis['LR_kappa'] = kappas_lr
pairwise_analysis['UMAP_LR_accuracy'] = accuracies_ulr
pairwise_analysis['UMAP_LR_f1score'] = f1scores_ulr
pairwise_analysis['UMAP_LR_kappa'] = kappas_ulr

print(pairwise_analysis)
path = os.path.join(directory,'Pairwise_Analysis_Stats.csv')
pairwise_analysis.to_csv(path,index= False)


BFcol_BFgam
4836295
1.) Selecting Mega Base Pairs
MBP selected. Retained Matrix =  (3651720, 334)
2.) Filtering Rare Allels
Number of SNPs removed =  3448792
Retaining =  202928
3.) Performing LD Pruning


In [98]:
pairwise_analysis.head(5)

,Count,Population_pair,Samples,Initial_SNP,MAF_filter_SNP,Filtered_SNP,LR_accuracy,LR_f1score,LR_kappa,UMAP_LR_accuracy,UMAP_LR_f1score,UMAP_LR_kappa
0,NaN,BFcol_BFgam,334,4836295,202928,119841,1.0,1.0,1.0,1.0,1.0,1.0
